# Notebook Experimental

Dieses Notebook kann dazu verwendet werden, außerhalb des durch die anderen Notebooks vorgegebenen Prozesses Code zu schreiben und zu testen - ohne dabei direkten Einfluss auf den Prozess zu nehmen. Typische Anwendungsbeispiele finden Sie unten aufgelistet.

In [1]:
%load_ext autoreload
%autoreload 2

---

#### Anzeigen des Intermediate Datensatzes

In [2]:
import pandas as pd

df = pd.read_feather('../data/intermediate/twitter_tweets_intermediate.feather')
df.head(4)

,url,date,rawContent
0,https://twitter.com/_Bob_S/status/164159110508...,2023-03-30 23:59:00,Govt IT security isnt 'a nice thing to do': it...
1,https://twitter.com/WYSIWYGVentures/status/164...,2023-03-30 23:59:00,ISC West 2023: Cyberattackers Are Targeting Ph...
2,https://twitter.com/HackerAran7/status/1641591...,2023-03-30 23:59:00,What’s the hack. #stem #science #stemeducation...
3,https://twitter.com/bytefeedai/status/16415909...,2023-03-30 23:59:00,BuzzFeed Is Using AI To Write SEO-Bait Travel ...


#### Generierung der list_parameter_combinations.pkl

Die Erstellung dieser Datei ist notwenig, um das Hyperparameter Tuning headless durchführen zu können

In [3]:
# %%script false
from src import utils
import itertools

num_topics = [i for i in range(4, 42)]
alpha = ['symmetric', 'asymmetric'] + [round(i*0.1, 1) for i in range(1, 10)]
eta = ['symmetric', 'auto'] + [round(i*0.1, 1) for i in range(1, 10)]
passes = [i for i in range(3, 11)]

combinations = list(itertools.product(num_topics, alpha, eta, passes))
combinations_dict = [{'num_topics': item[0], 'alpha': item[1], 'eta': item[2], 'passes': item[3]} for item in combinations]
utils.safe_as_pkl(combinations_dict, '../data/modeling/list_parameter_combinations.pkl')

[3, 4, 5, 6, 7, 8, 9, 10]

---